In [1]:
import math
import cv2
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator

In [2]:
# Путь к весам
model = YOLO("../weights/yolov10n.pt")

# Путь к видеофайлу
cap = cv2.VideoCapture("../test_videos/snowboarding.mp4")

# Получение параметров видео: ширина, высота и FPS
w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))

# Выходной видеофайл с настройками кодека
out = cv2.VideoWriter("../output_videos/visioneye_distance_calculation_output.mp4", cv2.VideoWriter_fourcc(*"MJPG"), fps, (w, h))

# Параметры вычисления
center_point = (0, h)  # Центр экрана
pixel_per_meter = 10   # Соотношение пикселей на метр

# Цвета для аннотаций
txt_color = (0, 0, 0)  # Цвет текста
txt_background = (255, 255, 255)  # Цвет фона текста
bbox_clr = (255, 0, 255)  # Цвет рамки

OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [3]:
while True:
    ret, im0 = cap.read()
    if not ret:
        print("Кадр видео пустой или обработка видео прошла успешно.")
        break

    # Создаем аннотатор для кадра
    annotator = Annotator(im0, line_width=2)

    # Обрабатываем кадр моделью YOLO
    results = model.track(im0, persist=True)
    boxes = results[0].boxes.xyxy.cpu()

    if results[0].boxes.id is not None:
        track_ids = results[0].boxes.id.int().cpu().tolist()

        for box, track_id in zip(boxes, track_ids):
            annotator.box_label(box, label=str(track_id), color=bbox_clr)
            annotator.visioneye(box, center_point)

            # Вычисляем центр рамки
            x1, y1 = int((box[0] + box[2]) // 2), int((box[1] + box[3]) // 2)

            # Вычисляем расстояние до точки внизу экрана
            distance = (math.sqrt((x1 - center_point[0]) ** 2 + (y1 - center_point[1]) ** 2)) / pixel_per_meter

            # Добавляем текст расстояния на кадр
            text_size, _ = cv2.getTextSize(f"Distance: {distance:.2f} m", cv2.FONT_HERSHEY_SIMPLEX, 1.2, 3)
            cv2.rectangle(im0, (x1, y1 - text_size[1] - 10), (x1 + text_size[0] + 10, y1), txt_background, -1)
            cv2.putText(im0, f"Distance: {distance:.2f} m", (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 1.2, txt_color, 3)

    # Сохраняем обработанный кадр
    out.write(im0)
    cv2.imshow("Visioneye distance calculation", im0)

    # Выход при нажатии клавиши "q"
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Освобождение ресурсов
out.release()
cap.release()
cv2.destroyAllWindows()


0: 640x384 1 person, 148.3ms
Speed: 2.6ms preprocess, 148.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 143.1ms
Speed: 2.5ms preprocess, 143.1ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 187.7ms
Speed: 2.0ms preprocess, 187.7ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 155.0ms
Speed: 2.2ms preprocess, 155.0ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 117.8ms
Speed: 1.8ms preprocess, 117.8ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 194.1ms
Speed: 6.5ms preprocess, 194.1ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 113.4ms
Speed: 2.0ms preprocess, 113.4ms inference, 0.1ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 person, 222.2ms
Speed: 3.7ms preprocess, 222.2ms inference, 0.2ms postprocess per image at